# Lab 2. Solar Panel Instructions

## Introduction

In this notebook we show you how to create your second sub-agent on Amazon Bedrock Agents.

This agent contains instructions on how to install and to do maintenance on Solar Panels, where customers can ask the agent to return these information from an [Amazon Bedrock Knowledge Base](https://aws.amazon.com/bedrock/knowledge-bases/).

To equip foundation models (FMs) with up-to-date and proprietary information, organizations use Retrieval Augmented Generation (RAG), a technique that fetches data from company data sources and enriches the prompt to provide more relevant and accurate responses. 

Amazon Bedrock Knowledge Bases is Bedrock's fully managed capability that helps you implement the entire RAG workflow from ingestion to retrieval and prompt augmentation without having to build custom integrations to data sources and manage data flows.

In the context of our agent, if the answer to a question is not present on the Knowledge Base, customers can ask agent to create a support ticket, to get a human in the loop to help with their questions.

The following represents the piece of architecture that will be built on this module.

![Architecture](img/solar_panel_agent.png)

## Setup

Firstly, you are going to install boto3 dependencies from pip. Make sure to have version superior of **1.35.45**

In [ ]:
# Install Dependencies from local package
!pip install ../boto3/controlplane/botocore-1.35.55-py3-none-any.whl \
    ../boto3/controlplane/boto3-1.35.55-py3-none-any.whl \
    ../boto3/controlplane/awscli-1.35.21-py3-none-any.whl --force-reinstall --no-cache

One component of a Knowledge Base is a Vector Database. Vector database is a kind of database that is optimized to store and search with vectors, making RAG architectures possible and faster. 

Amazon Bedrock Knowledge Bases is powered by one vector database. 

For this lab we are going to create one collection using [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/). If you want to know more, [this link](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-setup.html) contains other supported vector engines for Amazon Bedrock Knowledge Bases.

Once you select your vector database, you need to populate it with documents indexed from a data storage. Amazon Bedrock Kwnoledge Bases allow you to use different [data connectors](https://docs.aws.amazon.com/bedrock/latest/userguide/data-source-connectors.html) to populate your vector database. For this lab we will store our data in an Amazon S3 bucket and connect it with our knowledge base.

Now, let's install opensearch-py to interact with Opensearch engine.

In [ ]:
!pip install opensearch-py retrying --force-reinstall --no-cache

Restart kernel for packages to take effect

In [ ]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

Check if your boto3 version is superior than **1.35.45**

In [ ]:
!pip freeze | grep boto3

## Creating Agent

On this section we declare global variables that will be act as helpers during entire notebook and we will start to create out second agent.

In [1]:
import boto3
import os
import sys

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0',
    'anthropic.claude-3-5-sonnet-20240620-v1:0'
]

In [2]:
solar_agent_name = f"solar-p-{account_id}"

solar_lambda_name = f"fn-solar-p-{account_id}"

solar_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{solar_agent_name}'

dynamodb_table = f"{solar_agent_name}-table"
dynamodb_pk = "customer_id"
dynamodb_sk = "ticket_id"

dynamoDB_args = [dynamodb_table, dynamodb_pk, dynamodb_sk]

knowledge_base_name = f'{solar_agent_name}-kb'
suffix = f"{region}-{account_id}"

knowledge_base_description = "KB containing solar panel instructions for installation and maintenance"
bucket_name = f'{solar_agent_name}-{suffix}'

On following section, we're adding `agents.py`and `knowledge_bases.py` on Python path, so files can be recognized and invoked.

In [3]:
# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

parent_dir = os.path.dirname(current_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

Now, you're going to import from helper files `agents.py` and `knowledge_bases.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:
- `create_agent`: Create a new agent and respective IAM roles
- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent
- `create_agent_alias`: Create an alias for this agent
- `invoke`: Execute agent

On `knowledge_bases.py`:
- `create_or_retrieve_knowledge_base`: Create Knowledge Base on Amazon Bedrock if it doesn't exist or get info about previous created.
- `synchronize_data`: Read files on S3, convert text info into vectors and add that information on Vector Database.

In [4]:
from agent import AgentsForAmazonBedrock
from knowledge_bases import KnowledgeBasesForAmazonBedrock

agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

us-east-1


## Create and Load Knowledge Base

On this section, you're going to create a Amazon Bedrock Knowledge Base and ingest data on it.

In the next steps we will generate the data used to populate the knowledge base. It will be composed of instructions on how to handle a solar panel

**This creation process takes around 5min.**

In [5]:
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

Creating KB solar-p-471112746845-kb
Step 1 - Creating or retrieving solar-p-471112746845-us-east-1-471112746845 S3 bucket for Knowledge Base documents
Creating bucket solar-p-471112746845-us-east-1-471112746845
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_508) and Policies
Step 3 - Creating OSS encryption, network and data access policies
Step 4 - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '319',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Tue, 26 Nov 2024 04:20:42 '
                                                 'GMT',
                                         'x-amzn-requestid': '86a40f79-9c15-4d5c-92af-b9dbf5e31503'},
                        'HTTPStatusCode': 200,
                        

[2024-11-26 04:22:13,343] p7199 {base.py:258} INFO - PUT https://s3asy451r3o9vnyqwo14.us-east-1.aoss.amazonaws.com:443/solar-p-471112746845-kb-index-508 [status:200 request:0.533s]



Creating index:
{ 'acknowledged': True,
  'index': 'solar-p-471112746845-kb-index-508',
  'shards_acknowledged': True}
Step 6 - Creating Knowledge Base
{'type': 'VECTOR', 'vectorKnowledgeBaseConfiguration': {'embeddingModelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-text-v2:0'}}
{ 'createdAt': datetime.datetime(2024, 11, 26, 4, 23, 13, 447283, tzinfo=tzlocal()),
  'description': 'KB containing solar panel instructions for installation and '
                 'maintenance',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-east-1:471112746845:knowledge-base/TEU7AJU8YL',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'TEU7AJU8YL',
  'name': 'solar-p-471112746845-kb',
  'roleArn': 'arn:aws:iam::471112746845:role/AmazonBedrockExecutionRoleForKnowledgeBase_508',
  'status': 'CREA

### Create Synthetic Data to Load on S3

Instead of get data elsewhere, you're going to generate data, using a LLM on Amazon Bedrock.
This fake data that will be generated, will be uploaded into a S3 bucket and then added into an Amazon Bedrock Knowledge Base.

In [6]:
path = "kb_documents"
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:
   # Create a n ew directory if it does not exist
   os.makedirs(path)
   print("The {} directory was created!".format(path))
else:
   print("The {} directory already exists!".format(path))

The kb_documents directory already exists!


Creating helper methods to invoke LLM on Bedrock and to write a local file using Python

In [7]:
def invoke_bedrock_generate_energy_files(prompt):
    message_list = []

    initial_message = {
        "role": "user",
        "content": [
            { "text": prompt } 
        ],
    }

    message_list.append(initial_message)

    response = bedrock_client.converse(
        modelId=agent_foundation_model[0],
        messages=message_list,
        inferenceConfig={
            "maxTokens": 500,
            "temperature": 0
        },
    )

    return response['output']['message']

def write_file(file_name, content):
    f = open(file_name, 'w')
    f.write(content)
    f.close

Generating one file with instructions on how to install a solar panel.

In [8]:
text_generation_energy_instructions = '''
    You will be act as an expert on clean energy.
    You will generate a step-by-step on how to install a solar panel at home.
    Just answer in a numbered list.
'''

solar_energy_file_name = 'solar-panel-instructions.txt'

response_message = invoke_bedrock_generate_energy_files(text_generation_energy_instructions)

write_file('{}/{}'.format(path, solar_energy_file_name),response_message['content'][0]['text'])

### Generating data prompt
Generating another file with instructions on how to do maintenance on a solar panel.

In [9]:
text_generation_energy_instructions = '''
    You will be act as an expert on clean energy.
    You will generate a step-by-step instructions on how to do maintenance in a solar panel installed at a regular home.
    Just answer in a numbered list.
'''

solar_energy_file_name = 'solar-panel-maintenance.txt'

response_message = invoke_bedrock_generate_energy_files(text_generation_energy_instructions)

write_file('{}/{}'.format(path, solar_energy_file_name),response_message['content'][0]['text'])


### Uploading data to S3
Uploading generated files into S3 Bucket.

In [10]:
def upload_directory(path, bucket_name):
    for root,dirs,files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root,file)
            print(f"uploading file {file_to_upload} to {bucket_name}")
            s3_client.upload_file(file_to_upload,bucket_name,file)

### Synchronizing Knowledge Base
Now that the data is available in the s3 bucket, let's synchronize it to our knowledge base

In [11]:
upload_directory("kb_documents", bucket_name)

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

uploading file kb_documents/solar-panel-instructions.txt to solar-p-471112746845-us-east-1-471112746845
uploading file kb_documents/solar-panel-maintenance.txt to solar-p-471112746845-us-east-1-471112746845
{ 'dataSourceId': 'LZ9LGIA7WR',
  'ingestionJobId': 'OKNYBE9F9P',
  'knowledgeBaseId': 'TEU7AJU8YL',
  'startedAt': datetime.datetime(2024, 11, 26, 4, 24, 49, 220746, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2024, 11, 26, 4, 24, 49, 220746, tzinfo=tzlocal())}
{ 'dataSourceId': 'LZ9LGIA7WR',
  'ingestionJobId': 'OKNYBE9F9P',
  'knowledgeBaseId': 'TEU7AJU8YL',
  'startedAt': datetime.da

## Creating Agent

Create the Solar Panel agent that will have a Knowledge Base and also a Lambda action group to handle exception workflow (when information is not found on KB).

For this agent we will use the following instructions:
```
You are a Solar Energy Assistant that helps customers with solar panel installation and maintenance guidance.

Your capabilities include:
1. Providing installation instructions
2. Providing maintenance procedures
3. Troubleshooting common issues
4. Creating support tickets for specialist assistance

Core behaviors:
1. Always use available information before asking customers for additional details
2. Maintain a professional yet approachable tone
3. Provide clear, direct answers
4. Present technical information in an easy-to-understand manner
5. NEVER invent information not available in your knowledge base

Support ticket protocol:
- Only generate tickets for specialist-level issues
- Respond exclusively with case ID when creating tickets
- Decline providing specialist advice beyond your scope

Response style:
- Be helpful and solution-oriented
- Use clear, practical language
- Focus on actionable guidance
- Maintain natural conversation flow
- Be concise yet informative
- Do not add extra information not required by the user
```
We will also connect a knowledge base with the information about solar panels

```
Access the knowledge base when customers ask about to install and maintain solar panels
```
And we will make the following tool available to the agent:
- `open_ticket`: to open new support tickets
- `get_ticket_status`: to get the current status of an existing ticket

In [12]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

In [13]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': 'Access the knowledge base when customers ask about to install and maintain solar panels'
}

In [14]:
agent_instruction = """You are a Solar Energy Assistant that helps customers with solar panel installation and maintenance guidance.

Your capabilities include:
1. Providing installation instructions
2. Offering maintenance procedures
3. Troubleshooting common issues
4. Creating support tickets for specialist assistance

Core behaviors:
1. Always use available information before asking customers for additional details
2. Maintain a professional yet approachable tone
3. Provide clear, direct answers
4. Present technical information in an easy-to-understand manner

Support ticket protocol:
- Only generate tickets for specialist-level issues
- Respond exclusively with case ID when creating tickets
- Decline providing specialist advice beyond your scope

Response style:
- Be helpful and solution-oriented
- Use clear, practical language
- Focus on actionable guidance
- Maintain natural conversation flow
- Be concise yet informative
- Do not add extra information not required by the user"""

agent_description = """You are a solar energy helper bot. 
    You can retrieve information on how to install and do maintenance on solar panels"""

solar_agent_id = agents.create_agent(
    solar_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn]
)

solar_agent_id

'9COM2BU57I'

In [15]:
agents.associate_kb_with_agent(solar_agent_id, kb_config['kb_instruction'], kb_config['kb_id'])

### Creating Action Group

On this session, we're going create an action group to handle support tickets and associate it with our agent. To do so, we will first create a Lambda function code to fulfill the execution of the agent's actions Next we will define the actions available actions that an agent can take using function details. Similar to the previous agent, you can also define the actions available using OpenAPI Schema.

#### Creating Lambda function
First let's create the lambda function

In [16]:
%%writefile solar_energy.py
import os
import json
import uuid
import boto3

from boto3.dynamodb.conditions import Key, Attr

dynamodb_resource = boto3.resource('dynamodb')
dynamodb_table = os.getenv('dynamodb_table')
dynamodb_pk = os.getenv('dynamodb_pk')
dynamodb_sk = os.getenv('dynamodb_sk')

def get_named_parameter(event, name):
    try:
        return next(item for item in event['parameters'] if item['name'] == name)['value']
    except:
        return None
    
def populate_function_response(event, response_body):
    return {'response': {'actionGroup': event['actionGroup'], 'function': event['function'],
                'functionResponse': {'responseBody': {'TEXT': {'body': str(response_body)}}}}}

def put_dynamodb(table_name, item):
    table = dynamodb_resource.Table(table_name)
    resp = table.put_item(Item=item)
    return resp

def read_dynamodb(table_name: str, 
                   pk_field: str,
                   pk_value: str,
                   sk_field: str=None, 
                   sk_value: str=None):
    try:
        table = dynamodb_resource.Table(table_name)
        # Create expression
        if sk_value:
            key_expression = Key(pk_field).eq(pk_value) & Key(sk_field).begins_with(sk_value)
        else:
            key_expression = Key(pk_field).eq(pk_value)

        query_data = table.query(
            KeyConditionExpression=key_expression
        )
        
        return query_data['Items']
    except Exception:
        print(f'Error querying table: {table_name}.')

def open_ticket(customer_id, msg):
    ticket_id = str(uuid.uuid1())
    item = {
        'ticket_id': ticket_id,
        'customer_id': customer_id,
        'description': msg,
        'status': 'created'
    }
    resp = put_dynamodb(dynamodb_table, item)
    print(resp)
    return "Thanks for contact customer {}! Your support case was generated with ID: {}".format(
        customer_id, ticket_id
    )

def get_ticket_status(customer_id,
                      ticket_id: str=None):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk,
                         customer_id,
                         dynamodb_sk,
                         ticket_id)

def lambda_handler(event, context):
    print(event)
    
    # name of the function that should be invoked
    function = event.get('function', '')

    # parameters to invoke function with
    parameters = event.get('parameters', [])
    customer_id = get_named_parameter(event, "customer_id")

    if function == 'open_ticket':
        msg = get_named_parameter(event, "msg")
        result = open_ticket(customer_id, msg)
    elif function == 'get_ticket_status':
        ticket_id = get_named_parameter(event, "ticket_id")
        result = get_ticket_status(customer_id, ticket_id)
    else:
        result = f"Error, function '{function}' not recognized"

    response = populate_function_response(event, result)
    print(response)
    return response

Overwriting solar_energy.py


#### Defining available actions
Now it's time to define the actions that can be taken by the agent

In [17]:
functions_def =[
    {
        "name": "open_ticket",
        "description": """Create a ticket to get help with information related with solar panel or clean energy""",
        "parameters": {
                        "customer_id": {
                            "description": "Unique customer identifier",
                            "required": True,
                            "type": "string"
                        },
                        "msg": {
                            "description": "The reason why customer is opening a ticket",
                            "required": True,
                            "type": "string"
                        }
                    }
    },
    {
        "name": "get_ticket_status",
        "description": """get the status of an existing ticket""",
        "parameters": {
                        "customer_id": {
                            "description": "Unique customer identifier",
                            "required": True,
                            "type": "string"
                        },
                        "ticket_id": {
                            "description": "Unique ticket identifier",
                            "required": False,
                            "type": "string"
                        }
                    }
    }
]

#### Associating action group to agent
Finally, we can associate a new action group with our previously created agent

In [18]:
resp = agents.add_action_group_with_lambda(
            agent_name=solar_agent_name,
            lambda_function_name=solar_lambda_name,
            source_code_file="solar_energy.py",
            agent_functions=functions_def,
            agent_action_group_name="solar_energy_actions",
            agent_action_group_description="Function to open an energy ticket for a user or get status from an opened ticket",
            dynamo_args=dynamoDB_args
)

## Testing Agent

Now, let's run some tests on the agent we just created to make sure it's working. To do so we will use our test alias: `TSTALIASID` which allows you to invoke a draft version of your agent

### Testing maintainance question
First let's ask a question related to maintaining an existing solar panel

In [19]:
%%time
print(
    agents.invoke(
        "how can I check if my solar panel eletrical consumption is compliant with energy rules?", 
        solar_agent_id
    )
)



To check if your solar panel electrical consumption is compliant with energy rules, you should:

1. Obtain an inspection from your local authorities after installing the solar panel system. This inspection will ensure the system meets all safety and code requirements for electrical installations in your area.

2. Monitor the system's performance regularly by checking the energy output and comparing it to the expected output. If there is a significant drop in performance, it may indicate an issue that needs further investigation to ensure compliance.

3. Keep a log of all maintenance activities, including any issues or repairs made. This record can help demonstrate that you are properly maintaining the system and addressing any potential compliance issues.

4. If you encounter any issues or problems that you are not comfortable addressing, or if the system requires more complex maintenance or repairs, consider hiring a professional solar panel technician or contractor to ensure the sy

### Testing installation question
Next let's ask a question related to installing a new solar panel

In [28]:
%%time
print(
    agents.invoke(
        "how can I install a solar pannel at home?", 
        solar_agent_id
    )
)



To install solar panels at home, follow these key steps:

1. Assess your energy needs and determine the size of the solar panel system required for your home.
2. Obtain necessary permits and approvals from your local authorities. 
3. Choose a suitable location for the solar panels, preferably a south-facing roof or an area with minimal shade.
4. Prepare the installation site by clearing obstructions and ensuring a stable mounting surface.
5. Install the racking system or mounting hardware according to manufacturer's instructions and local codes.
6. Mount the solar panels onto the racking system, ensuring proper alignment and secure connections.
7. Install the inverter to convert the direct current (DC) from the panels into alternating current (AC) for household use.
8. Connect the solar panels to the inverter using appropriate cables and connectors.
9. Install necessary electrical components like circuit breakers, meters, and disconnect switches to integrate the system with your home

### Testing personalized support
Let's now create a support ticket. To do so we will pass our customer id to trigger the support ticket creation process

In [20]:
%%time
print(
    agents.invoke(
        "Can I get support to install my solar panel? My customer id is 1", 
        solar_agent_id
    )
)

a89839d6-abae-11ef-b5e9-c5a2bed3bc09
CPU times: user 2.39 ms, sys: 2.37 ms, total: 4.77 ms
Wall time: 9.49 s


### Testing getting support ticket details
Now let's get the details for our support ticket. For that you will need to provide the ticket id just generated in the previous query to `create_ticket`

In [21]:
created_ticket = "a89839d6-abae-11ef-b5e9-c5a2bed3bc09"

In [22]:
%%time
print(
    agents.invoke(
        "Can I get details on my ticket? My customer id is 1 and my ticket id is {}".format(created_ticket), 
        solar_agent_id
    )
)

Your ticket with ID a89839d6-abae-11ef-b5e9-c5a2bed3bc09 has a status of 'created'. The description is 'Request for support to install solar panel system'.
CPU times: user 2.47 ms, sys: 3.38 ms, total: 5.84 ms
Wall time: 14.9 s


### Testing get open support tickets
Let's also check if our agent can get the details for more than one open case. To do so we will first create a new support case

In [23]:
%%time
print(
    agents.invoke(
        "Can I get support to review my solar panel consumption? My customer id is 1", 
        solar_agent_id
    )
)

db46116f-abae-11ef-8445-c5a2bed3bc09
CPU times: user 4.44 ms, sys: 0 ns, total: 4.44 ms
Wall time: 7.42 s


Next we will request the overview of all open tickets

In [24]:
%%time
print(
    agents.invoke(
        "Can I get all tickets that I have? My customer id is 1", 
        solar_agent_id
    )
)

Here are the tickets associated with your customer ID 1:

Ticket ID: a89839d6-abae-11ef-b5e9-c5a2bed3bc09
Description: Request for support to install solar panel system  
Status: created

Ticket ID: db46116f-abae-11ef-8445-c5a2bed3bc09
Description: Request for support to review solar panel consumption
Status: created
CPU times: user 2.55 ms, sys: 1.98 ms, total: 4.53 ms
Wall time: 7.88 s


## Create alias

As you can see, you can use your agent with the `TSTALIASID` to complete tasks. 
However, for multi-agents collaboration it is expected that you first test your agent and only use it once it is fully functional. 
Therefore to use an agent as a sub-agent in a multi-agent collaboration you first need to create an agent alias and connect it to a new version. 

Since we've tested and validated our agent, let's now create an alias for it:

In [25]:
agents_boto3 = boto3.client(
    'bedrock-agent',
    region_name=region
)

solar_agent_alias_resp = agents_boto3.create_agent_alias(
    agentId=solar_agent_id,
    agentAliasName='v1'
)

Store environment variables to be used on next notebooks.

In [26]:
solar_agent_arn = agents.get_agent_arn_by_name(solar_agent_name)
solar_agent_alias_arn = solar_agent_alias_resp['agentAlias']['agentAliasArn']
solar_agent_alias_id = solar_agent_alias_resp['agentAlias']['agentAliasId']
solar_kb = knowledge_base_name
solar_dynamodb = dynamodb_table

%store solar_agent_arn
%store solar_agent_alias_arn
%store solar_agent_alias_id
%store solar_lambda_name
%store solar_agent_name
%store solar_agent_id
%store solar_kb
%store solar_dynamodb

Stored 'solar_agent_arn' (str)
Stored 'solar_agent_alias_arn' (str)
Stored 'solar_agent_alias_id' (str)
Stored 'solar_lambda_name' (str)
Stored 'solar_agent_name' (str)
Stored 'solar_agent_id' (str)
Stored 'solar_kb' (str)
Stored 'solar_dynamodb' (str)


In [ ]:
solar_agent_arn, solar_agent_alias_arn, solar_agent_alias_id

### [Optional] Clean Up
We will clean up the resources created after the lab 4. However, we will also let some commented out code here in case you want to clean up intermediate results

In [ ]:
# agents.delete_lambda(solar_lambda_name, dynamoDB_table=dynamodb_table)

In [ ]:
# agents_boto3.delete_agent_alias(agentAliasId=solar_agent_alias_id, agentId=solar_agent_id)

In [ ]:
# agents.delete_agent(solar_agent_name)

In [ ]:
# kb.delete_kb(knowledge_base_name)

## Next Steps
Congratulations! We've now created a solar panel agent. Next we will create our peak loader manager agent